# Line Break Chart Backtest

In [ ]:
pip install "yfinance[nospam]" stocktrends matplotlib pandas requests_cache

In [ ]:
import yfinance as yf
import pandas as pd
import stocktrends as st
import numpy as np
from concurrent.futures import ThreadPoolExecutor, as_completed
import traceback
import requests_cache

session = requests_cache.CachedSession('yfinance.cache')
session.headers['User-agent'] = 'Mozilla/5.0 (compatible; MSIE 7.0; Windows; U; Windows NT 10.4; Win64; x64; en-US Trident/4.0)'

results = []

tickers_txt_path = 'us_stock.txt'

try:
    with open(tickers_txt_path, 'r') as file:
        tickers = [line.strip() for line in file if line.strip()]
except Exception as e:
    print(f"Error reading tickers from text file: {e}")
    tickers = []

def is_uptrend(data_lb):
    if len(data_lb) < 4:
        return False

    current_bar = data_lb.iloc[-1]
    last_bar = data_lb.iloc[-2]
    second_last_bar = data_lb.iloc[-3]
    third_last_bar = data_lb.iloc[-4]

    condition1 = (
        current_bar['uptrend'] and
        not last_bar['uptrend'] and
        second_last_bar['uptrend'] and
        third_last_bar['uptrend']
    )

    condition2 = (
        current_bar['uptrend'] and
        data_lb.iloc[-1]['uptrend'] and
        current_bar['close'] > data_lb.iloc[-2]['close'] and
        current_bar['close'] > data_lb.iloc[-3]['close'] and
        current_bar['close'] > data_lb.iloc[-4]['close']
    )

    return condition1 or condition2

def ehlers_instantaneous_trendline(source, alpha):
    itrend = np.zeros_like(source)
    trigger = np.zeros_like(source)

    for i in range(len(source)):
        if i < 7:
            itrend[i] = (source[i] + (2 * source[i - 1]) + source[i - 2]) / 4
        else:
            itrend[i] = (
                ((alpha - (alpha ** 2 / 4)) * source[i]) +
                (0.5 * alpha ** 2 * source[i - 1]) -
                ((alpha - (0.75 * alpha ** 2)) * source[i - 2]) +
                (2 * (1 - alpha) * itrend[i - 1]) -
                ((1 - alpha) ** 2 * itrend[i - 2])
            )

        if i >= 2:
            trigger[i] = (2 * itrend[i]) - itrend[i - 2]

    return itrend, trigger

def process_ticker(ticker):
    try:
        print(f"Processing {ticker}")
        data = yf.download(ticker, period="6mo", session=session)

        if data.empty:
            print(f"No data returned for ticker {ticker}")
            return None

        data.reset_index(inplace=True)
        data.columns = [col.lower() for col in data.columns]

        lb = st.indicators.LineBreak(data)
        lb.line_number = 3
        data_lb = lb.get_ohlc_data()

        if data_lb.empty:
            print(f"No LineBreak data for ticker {ticker}")
            return None

        uptrend = is_uptrend(data_lb)

        if uptrend:
            itrend, _ = ehlers_instantaneous_trendline(data_lb["close"].values, 0.07)
            ma_20 = data_lb["close"].rolling(window=20).mean()
            g = round(itrend[-1] / itrend[-2] - 1, 4)
            price = round(data['adj close'].iloc[-1], 2)
            min_open = min(data_lb['open'][-4:])
            min_close = min(data_lb['close'][-4:])
            sl = round(min(min_open, min_close, itrend[-1]), 2)
            risk = round((price - sl) / price, 4)
            rr = round(g / risk, 4) if risk != 0 else np.nan
            monthly_return = round(data['adj close'].iloc[-1] / data['adj close'].iloc[-22] - 1, 2)

            if itrend[-1] > itrend[-2] and \
                ma_20.iloc[-1] > ma_20.iloc[-2] and \
                rr > 0.15 and \
                data['adj close'].iloc[-1] / data['adj close'].iloc[-22] >= 1.05 and \
                data['adj close'].iloc[-1] > data_lb['open'].iloc[-1] and \
                (data['adj close'].iloc[-1] - data_lb['open'].iloc[-1]) / \
                    (data_lb['close'].iloc[-1] - data_lb['open'].iloc[-1]) >= 0.8 and \
                monthly_return > 0.10:

                result = {
                    'ticker': ticker,
                    'g': g,
                    'price': price,
                    'sl': sl,
                    'risk': risk,
                    'monthly_return': monthly_return,
                    'rr': rr
                }

                return result
        return None
    except Exception as e:
        print(f"Error processing {ticker}: {e}")
        print("Traceback:", traceback.format_exc())
        return None

with ThreadPoolExecutor(max_workers=1) as executor:
    future_to_ticker = {executor.submit(process_ticker, ticker): ticker for ticker in tickers}

    for future in as_completed(future_to_ticker):
        result = future.result()
        if result:
            results.append(result)

df_results = pd.DataFrame(results)

df_results = df_results.sort_values(by='monthly_return', ascending=False)

csv_file_path = 'filtered_stocks.csv'
df_results.to_csv(csv_file_path, index=False)

print(f"Filtered stocks saved to {csv_file_path}")